## Data Processing

In [3]:
import numpy as np
import pandas as pd
import sys
sys.path.append('c:/users/zhou/appdata/local/programs/python/python39/lib/site-packages')
from tabulate import tabulate
from pandas import Series,DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
import re



col_list = ['ProductId','ProductName','Branch','level_1','level_2','level_3','PricePerItem','PriceMode','HasMultiBuyDeal',
            'MultiBuyDeal','MultiBuyBasePrice','MultiBuyPrice','MultiBuyQuantity','PromoBadgeImageLabel']
products = pd.read_csv("D:/jupyter-workplace/classification_project/data_cleaning/data.csv", usecols=col_list, )
display(products.head())
print(products.shape)


,ProductId,ProductName,Branch,level_1,level_2,level_3,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
0,5000961-EA-000NW,Johnson's Baby Lotion,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
1,5000962-EA-000NW,Johnson's Baby Oil,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
2,5000963-EA-000NW,SMA Balanced Nutrition From Birth Infant Formula,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food,17.99,ea,False,NaN,17.99,17.99,1.0,Saver
3,5000965-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,2.99,ea,False,NaN,2.99,2.99,1.0,Saver
4,5000966-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,4.89,ea,False,NaN,4.89,NaN,NaN,NaN


(14612, 14)


In [4]:
# 去除相同产品数据
BranchSets = list(set(products['Branch']))
BranchSets.sort() 
data = pd.DataFrame()
index = 0
for Branch in BranchSets:
    sub_df = products.loc[products['Branch']==Branch]
    sub_df = sub_df.drop_duplicates(subset=['ProductName'], keep='first')
    data=data.append(sub_df)
    index = index + 1
data = data.drop(columns=["ProductId"])
data.columns = ['ProductName', 'Branch','level_1', 'level_2', 'level_3', 'PricePerItem','PriceMode','HasMultiBuyDeal',
            'MultiBuyDeal','MultiBuyBasePrice','MultiBuyPrice','MultiBuyQuantity','PromoBadgeImageLabel',]
data.to_csv(r'unique_product.csv', index = True, header=True)
display(data.head())
print(data.shape)

,ProductName,Branch,level_1,level_2,level_3,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
0,Johnson's Baby Lotion,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
1,Johnson's Baby Oil,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
2,SMA Balanced Nutrition From Birth Infant Formula,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food,17.99,ea,False,NaN,17.99,17.99,1.0,Saver
3,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,2.99,ea,False,NaN,2.99,2.99,1.0,Saver
5,Vaseline White Petroleum Jelly,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,3.49,ea,False,NaN,3.49,NaN,NaN,NaN


(12222, 13)


In [13]:
#重新整理特征项 
new_df = pd.read_csv("unique_product.csv")
col_order = ['ProductName','PricePerItem','level_1','level_2','level_3']
new_df = new_df[col_order]
new_df.index = range(0, len(new_df))
new_df.to_csv (r'unique_product_match.csv', index = True, header=True)
new_df.head()


,ProductName,PricePerItem,level_1,level_2,level_3
0,Johnson's Baby Lotion,4.89,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
1,Johnson's Baby Oil,4.89,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
2,SMA Balanced Nutrition From Birth Infant Formula,17.99,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food
3,Johnson's Baby Powder,2.99,"Baby, Toddler & Kids",Baby Care,Nappies & Changing
4,Vaseline White Petroleum Jelly,3.49,"Baby, Toddler & Kids",Baby Care,Nappies & Changing


In [14]:
#检查缺失值
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12222 entries, 0 to 14611
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductName           12222 non-null  object 
 1   Branch                12222 non-null  object 
 2   level_1               12222 non-null  object 
 3   level_2               12222 non-null  object 
 4   level_3               12222 non-null  object 
 5   PricePerItem          12222 non-null  float64
 6   PriceMode             12222 non-null  object 
 7   HasMultiBuyDeal       12222 non-null  bool   
 8   MultiBuyDeal          663 non-null    object 
 9   MultiBuyBasePrice     12222 non-null  float64
 10  MultiBuyPrice         4870 non-null   float64
 11  MultiBuyQuantity      4870 non-null   float64
 12  PromoBadgeImageLabel  5233 non-null   object 
dtypes: bool(1), float64(4), object(8)
memory usage: 1.2+ MB


In [15]:
#同一个商品有没有存在不同的类cat里
levels1 = data.groupby('ProductName')['level_1'].nunique().sort_values(ascending = False)
print(levels1)

ProductName
100% Kiwi Extra Virgin Olive Oil                          1
Pams Sunflower Seeds                                      1
Pams Strawberry Jam                                       1
Pams Strawberry Pro Bio Cultured Dairy Drink              1
Pams Stretch Crepe Bandage 75mm Wide                      1
                                                         ..
Hansells Chef's Strength Vanilla Bean Paste With Seeds    1
Hansells Citric Acid For Sour Flavour                     1
Hansells Cream Of Tartar Meringue Stabiliser              1
Hansells Greek Style Lite Yoghurt Mix                     1
e2 Orange Liquid Energy Fruit Drink                       1
Name: level_1, Length: 12222, dtype: int64


In [16]:
#(单价)价格均值
data.groupby('Branch')['PricePerItem'].agg(np.mean)

Branch
New World Mt Roskill    7.662477
Name: PricePerItem, dtype: float64

In [17]:
#level_1(第一类别)价格均值
data.groupby('level_1')['PricePerItem'].agg(np.mean)

level_1
Baby, Toddler & Kids            8.782429
Beer, Cider & Wine             15.797067
Chilled, Frozen & Desserts      6.276941
Drinks                          5.328174
Featured                        3.422533
Fresh Foods & Bakery           10.960293
Kitchen, Dining & Household     7.690276
Meal Spot                       3.547402
Pantry                          4.341755
Personal Care                   9.899113
Pets                            7.443921
Name: PricePerItem, dtype: float64

In [18]:
#第二类价格均值
data.groupby(['level_1','level_2'])['PricePerItem'].agg(np.mean)

level_1                      level_2                    
Baby, Toddler & Kids         Baby Care                       8.782429
Beer, Cider & Wine           Beer & Cider                   13.176781
                             Wine                           17.323245
Chilled, Frozen & Desserts   Cheese                          6.972391
                             Dairy & Eggs                    5.074668
                             Desserts                        6.436852
                             Frozen Foods                    6.833318
Drinks                       Cold Drinks                     4.511785
                             Hot Drinks                      6.349695
Featured                     $5 Fortnight                    3.422533
Fresh Foods & Bakery         Bakery                          5.285789
                             Bulk & Loose Foods             28.167388
                             Butchery                       14.643737
                             Deli

In [19]:
#第三类价格均值
data.groupby(['Branch','level_1','level_2','level_3'])['PricePerItem'].agg(np.mean)

Branch                level_1               level_2       level_3                    
New World Mt Roskill  Baby, Toddler & Kids  Baby Care     Baby Bathing & Skin Care        8.684118
                                                          Baby Dental Care                3.290000
                                                          Baby Formula & Toddler Food     7.158261
                                                          Baby Hair Care                  7.556667
                                                          Baby Health                     8.556667
                                                                                           ...    
                      Pets                  Pet Supplies  Dog Treats                      7.595116
                                                          Fish & Aquatic Pet Supplies     7.850000
                                                          Pet Accessories                 9.240000
                       

In [20]:
#价格方差
data.groupby('Branch')['PricePerItem'].agg(np.var)

Branch
New World Mt Roskill    55.163292
Name: PricePerItem, dtype: float64

In [21]:
#第一类商品价格方差
data.groupby(['Branch','level_1'])['PricePerItem'].agg(np.var)

Branch                level_1                    
New World Mt Roskill  Baby, Toddler & Kids            73.927319
                      Beer, Cider & Wine              96.606539
                      Chilled, Frozen & Desserts      12.172804
                      Drinks                           8.476645
                      Featured                         0.934595
                      Fresh Foods & Bakery           131.568510
                      Kitchen, Dining & Household     48.458395
                      Meal Spot                        5.489719
                      Pantry                           8.705358
                      Personal Care                   53.954540
                      Pets                            43.604253
Name: PricePerItem, dtype: float64

In [22]:
#第二类商品价格方差
data.groupby(['Branch','level_1','level_2'])['PricePerItem'].agg(np.var)

Branch                level_1                      level_2                    
New World Mt Roskill  Baby, Toddler & Kids         Baby Care                       73.927319
                      Beer, Cider & Wine           Beer & Cider                    58.209587
                                                   Wine                           112.741343
                      Chilled, Frozen & Desserts   Cheese                          13.463494
                                                   Dairy & Eggs                     4.749642
                                                   Desserts                        18.952810
                                                   Frozen Foods                    12.820865
                      Drinks                       Cold Drinks                      7.178314
                                                   Hot Drinks                       8.241362
                      Featured                     $5 Fortnight                     

In [23]:
#第三类商品价格方差
data.groupby(['Branch','level_1','level_2','level_3'])['PricePerItem'].agg(np.var)

Branch                level_1               level_2       level_3                    
New World Mt Roskill  Baby, Toddler & Kids  Baby Care     Baby Bathing & Skin Care        14.688449
                                                          Baby Dental Care                 0.796667
                                                          Baby Formula & Toddler Food    103.206454
                                                          Baby Hair Care                   3.446667
                                                          Baby Health                      5.663333
                                                                                            ...    
                      Pets                  Pet Supplies  Dog Treats                      16.706549
                                                          Fish & Aquatic Pet Supplies      2.718000
                                                          Pet Accessories                  1.125000
              

In [24]:
#第一类别数量
data.groupby('Branch')['level_1'].value_counts().head(50)

Branch                level_1                    
New World Mt Roskill  Pantry                         3750
                      Personal Care                  1658
                      Chilled, Frozen & Desserts     1324
                      Fresh Foods & Bakery           1227
                      Beer, Cider & Wine             1190
                      Kitchen, Dining & Household    1086
                      Drinks                          887
                      Pets                            454
                      Baby, Toddler & Kids            317
                      Meal Spot                       254
                      Featured                         75
Name: level_1, dtype: int64

In [25]:
#第二类别数量
data.groupby(['Branch','level_1'])['level_2'].value_counts().head(50)

Branch                level_1                      level_2                    
New World Mt Roskill  Baby, Toddler & Kids         Baby Care                       317
                      Beer, Cider & Wine           Wine                            752
                                                   Beer & Cider                    438
                      Chilled, Frozen & Desserts   Frozen Foods                    440
                                                   Dairy & Eggs                    392
                                                   Cheese                          276
                                                   Desserts                        216
                      Drinks                       Cold Drinks                     493
                                                   Hot Drinks                      394
                      Featured                     $5 Fortnight                     75
                      Fresh Foods & Bakery         

In [26]:
#第三类商品数量
data.groupby(['Branch','level_1','level_2'])['level_3'].value_counts().head(50)

Branch                level_1                     level_2       level_3                     
New World Mt Roskill  Baby, Toddler & Kids        Baby Care     Baby Formula & Toddler Food     161
                                                                Nappies & Changing               65
                                                                Baby Bathing & Skin Care         34
                                                                Nursing & Feeding                29
                                                                Baby Wipes                       12
                                                                Baby Dental Care                  7
                                                                Baby Hair Care                    6
                                                                Baby Health                       3
                      Beer, Cider & Wine          Beer & Cider  Lager                            87
       

In [27]:
#各项统计值(价格均值、方差、商品数量)
data.groupby('Branch')['PricePerItem'].describe()

,count,mean,std,min,25%,50%,75%,max
Branch,,,,,,,,
New World Mt Roskill,12222.0,7.662477,7.427199,0.15,3.49,5.0,9.0,129.9


In [28]:
#第一类别各项统计值
data.groupby(['Branch','level_1'])['PricePerItem'].describe()

count       mean  \
Branch               level_1                                          
New World Mt Roskill Baby, Toddler & Kids          317.0   8.782429   
                     Beer, Cider & Wine           1190.0  15.797067   
                     Chilled, Frozen & Desserts   1324.0   6.276941   
                     Drinks                        887.0   5.328174   
                     Featured                       75.0   3.422533   
                     Fresh Foods & Bakery         1227.0  10.960293   
                     Kitchen, Dining & Household  1086.0   7.690276   
                     Meal Spot                     254.0   3.547402   
                     Pantry                       3750.0   4.341755   
                     Personal Care                1658.0   9.899113   
                     Pets                          454.0   7.443921   

                                                        std   min   25%  \
Branch               level_1                                              
New World Mt Roskill Baby, Toddler & Kids          8.598100  0.99  1.99   
                     Beer, Cider & Wine            9.828863  1.79  9.99   
                     Chilled, Frozen & Desserts    3.488955  0.89  3.99   
                     Drinks                        2.911468  0.89  3.49   
                     Featured                      0.966744  1.29  2.79   
                     Fresh Foods & Bakery         11.470332  1.20  4.29   
                     Kitchen, Dining & Household   6.961206  0.15  3.96   
                     Meal Spot                     2.343015  0.99  1.99   
                     Pantry                        2.950484  0.50  2.59   
                     Personal Care                 7.345375  0.90  4.69   
                     Pets                          6.603352  1.00  2.49   

                                                    50%    75%     max  
Branch               level_1                                            
New World Mt Roskill Baby, Toddler & Kids          4.99  12.99   45.99  
                     Beer, Cider & Wine           14.99  18.99  120.99  
                     Chilled, Frozen & Desserts    5.74   7.79   39.90  
                     Drinks                        4.49   6.99   24.99  
                     Featured                      3.29   3.49    5.00  
                     Fresh Foods & Bakery          5.99  13.99  129.90  
                     Kitchen, Dining & Household   5.99   9.49   99.99  
                     Meal Spot                     2.79   4.49   14.90  
                     Pantry                        3.59   4.99   35.99  
                     Personal Care                 7.69  12.99   54.99  
                     Pets                          5.99   9.99   37.49

In [29]:
#第二类别各项统计值
data.groupby(['Branch','level_1','level_2'])['PricePerItem'].describe()

count  \
Branch               level_1                     level_2                               
New World Mt Roskill Baby, Toddler & Kids        Baby Care                     317.0   
                     Beer, Cider & Wine          Beer & Cider                  438.0   
                                                 Wine                          752.0   
                     Chilled, Frozen & Desserts  Cheese                        276.0   
                                                 Dairy & Eggs                  392.0   
                                                 Desserts                      216.0   
                                                 Frozen Foods                  440.0   
                     Drinks                      Cold Drinks                   493.0   
                                                 Hot Drinks                    394.0   
                     Featured                    $5 Fortnight                   75.0   
                     Fresh Foods & Bakery        Bakery                        323.0   
                                                 Bulk & Loose Foods            134.0   
                                                 Butchery                      198.0   
                                                 Deli, Salads & Cooked Meats   338.0   
                                                 Fruit & Vegetables            163.0   
                                                 Meal Kits                      13.0   
                                                 Seafood                        58.0   
                     Kitchen, Dining & Household Cleaning Products             296.0   
                                                 Garage & Outdoor               44.0   
                                                 Household                     524.0   
                                                 Laundry                       135.0   
                                                 Stationery & Entertainment     87.0   
                     Meal Spot                   Ready to Cook                  85.0   
                                                 Ready to Heat                 169.0   
                     Pantry                      Baking Supplies & Sugar       352.0   
                                                 Biscuits & Crackers           356.0   
                                                 Breakfast Cereals             192.0   
                                                 Canned & Prepared Foods       395.0   
                                                 Condiments & Dressings        311.0   
                                                 Confectionery                 346.0   
                                                 Jams, Honey & Spreads         166.0   
                                                 Pasta, Rice & Noodles         217.0   
                                                 Salad & Cooking Oils           72.0   
                                                 Sauces, Stock & Marinades     359.0   
                                                 Snack Foods                   506.0   
                                                 Spices & Seasonings           286.0   
                                                 World Foods                   192.0   
                     Personal Care               Beauty & Grooming            1164.0   
                                                 Health & Wellness             494.0   
                     Pets                        Pet Supplies                  454.0   

                                                                                   mean  \
Branch               level_1                     level_2                                  
New World Mt Roskill Baby, Toddler & Kids        Baby Care                     8.782429   
                     Beer, Cider & Wine          Beer & Cider                 13.176781   
                  

In [30]:
#第三类别各项统计值
data.groupby(['Branch','level_1','level_2','level_3'])['PricePerItem'].describe()

count  \
Branch               level_1              level_2      level_3                              
New World Mt Roskill Baby, Toddler & Kids Baby Care    Baby Bathing & Skin Care      34.0   
                                                       Baby Dental Care               7.0   
                                                       Baby Formula & Toddler Food  161.0   
                                                       Baby Hair Care                 6.0   
                                                       Baby Health                    3.0   
...                                                                                   ...   
                     Pets                 Pet Supplies Dog Treats                    43.0   
                                                       Fish & Aquatic Pet Supplies    5.0   
                                                       Pet Accessories                2.0   
                                                       Pet Treatments                17.0   
                                                       Small Animal Supplies          5.0   

                                                                                         mean  \
Branch               level_1              level_2      level_3                                  
New World Mt Roskill Baby, Toddler & Kids Baby Care    Baby Bathing & Skin Care      8.684118   
                                                       Baby Dental Care              3.290000   
                                                       Baby Formula & Toddler Food   7.158261   
                                                       Baby Hair Care                7.556667   
                                                       Baby Health                   8.556667   
...                                                                                       ...   
                     Pets                 Pet Supplies Dog Treats                    7.595116   
                                                       Fish & Aquatic Pet Supplies   7.850000   
                                                       Pet Accessories               9.240000   
                                                       Pet Treatments               20.154706   
                                                       Small Animal Supplies         9.350000   

                                                                                          std  \
Branch               level_1              level_2      level_3                                  
New World Mt Roskill Baby, Toddler & Kids Baby Care    Baby Bathing & Skin Care      3.832551   
                                                       Baby Dental Care              0.892562   
                                                       Baby Formula & Toddler Food  10.159058   
                                                       Baby Hair Care                1.856520   
                                                       Baby Health                   2.379776   
...                                                                                       ...   
                     Pets                 Pet Supplies Dog Treats                    4.087365   
                                                       Fish & Aquatic Pet Supplies   1.648636   
                                                       Pet Accessories               1.060660   
                                                       Pet Treatments               12.577954   
                                                       Small Animal Supplies         2.841303   

                                                                                     min  \
Branch               level_1              level_2      level_3                             
New World Mt Roskill Baby, Toddler & Kids Baby Care    Baby Bathing & Skin Care     1.79   
                                                       Baby Dental Care   

In [31]:
#第一类别商品数统计-降序

df_item_count = data.groupby(['Branch','level_1'])['ProductName'].count().sort_values(ascending =False).to_frame().reset_index()
display(df_item_count)

,Branch,level_1,ProductName
0,New World Mt Roskill,Pantry,3750
1,New World Mt Roskill,Personal Care,1658
2,New World Mt Roskill,"Chilled, Frozen & Desserts",1324
3,New World Mt Roskill,Fresh Foods & Bakery,1227
4,New World Mt Roskill,"Beer, Cider & Wine",1190
5,New World Mt Roskill,"Kitchen, Dining & Household",1086
6,New World Mt Roskill,Drinks,887
7,New World Mt Roskill,Pets,454
8,New World Mt Roskill,"Baby, Toddler & Kids",317
9,New World Mt Roskill,Meal Spot,254


In [32]:
#第二类别商品数统计-降序
data.groupby(['Branch','level_1','level_2'])['ProductName'].count().sort_values(ascending =False).to_frame().reset_index()

,Branch,level_1,level_2,ProductName
0,New World Mt Roskill,Personal Care,Beauty & Grooming,1164
1,New World Mt Roskill,"Beer, Cider & Wine",Wine,752
2,New World Mt Roskill,"Kitchen, Dining & Household",Household,524
3,New World Mt Roskill,Pantry,Snack Foods,506
4,New World Mt Roskill,Personal Care,Health & Wellness,494
5,New World Mt Roskill,Drinks,Cold Drinks,493
6,New World Mt Roskill,Pets,Pet Supplies,454
7,New World Mt Roskill,"Chilled, Frozen & Desserts",Frozen Foods,440
8,New World Mt Roskill,"Beer, Cider & Wine",Beer & Cider,438
9,New World Mt Roskill,Pantry,Canned & Prepared Foods,395


In [33]:
#第三类别商品数统计-降序
data.groupby(['Branch','level_1','level_2','level_3'])['ProductName'].count().sort_values(ascending =False).to_frame().reset_index()

,Branch,level_1,level_2,level_3,ProductName
0,New World Mt Roskill,Personal Care,Beauty & Grooming,Face & Lip Skin Care,216
1,New World Mt Roskill,Pets,Pet Supplies,Cat Food,216
2,New World Mt Roskill,Personal Care,Beauty & Grooming,Hair Care & Treatments,191
3,New World Mt Roskill,Personal Care,Beauty & Grooming,Hand & Body Care,179
4,New World Mt Roskill,Pantry,Spices & Seasonings,Dried Herbs,176
...,...,...,...,...,...
398,New World Mt Roskill,Fresh Foods & Bakery,Fruit & Vegetables,Vege Bags,1
399,New World Mt Roskill,"Chilled, Frozen & Desserts",Cheese,Deli Cabinet Cheeses,1
400,New World Mt Roskill,"Kitchen, Dining & Household",Laundry,Laundry Supplies,1
401,New World Mt Roskill,"Kitchen, Dining & Household",Laundry,Washing Machine Cleaners,1


In [34]:
#商品名称词频
data.ProductName.str.split(expand=True).stack().value_counts().head(50)

&            1743
Pams          755
Chicken       491
Chocolate     479
Cheese        379
Organic       348
Cream         343
Sauce         337
Food          331
Fresh         327
With          326
Free          311
Milk          287
Original      266
Fruit         266
Wattie's      252
Rice          250
Natural       232
Cat           219
The           218
Beef          217
White         215
Mix           213
In            207
Drink         203
Coffee        202
Coconut       201
Lemon         190
Value         187
Yoghurt       184
Pinot         182
Tea           180
Extra         179
Black         178
For           177
Oil           168
Bags          167
Juice         167
Red           167
Butter        164
Foods         160
Pasta         160
Tomato        157
Apple         156
Honey         156
Vanilla       153
Classic       150
Hair          147
Powder        146
Premium       145
dtype: int64

In [35]:
#生成最终真实数据
col_list = ['ProductName','level_1','level_2','level_3']
data = data[col_list]
display(data)
data.to_csv (r"product_classification.csv",index = False, header=True)
data = pd.read_csv("./product_classification.csv",usecols=col_list)
display(data)

,ProductName,level_1,level_2,level_3
0,Johnson's Baby Lotion,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
1,Johnson's Baby Oil,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
2,SMA Balanced Nutrition From Birth Infant Formula,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food
3,Johnson's Baby Powder,"Baby, Toddler & Kids",Baby Care,Nappies & Changing
5,Vaseline White Petroleum Jelly,"Baby, Toddler & Kids",Baby Care,Nappies & Changing
...,...,...,...,...
14605,Nood Wild-Caught Tuna Recipe In Gravy Premium ...,Pets,Pet Supplies,Cat Food
14606,Nood Grass-Fed Lamb Recipe In Gravy Premium Ca...,Pets,Pet Supplies,Cat Food
14607,Nood Cage-Free Chicken Recipe In Gravy Premium...,Pets,Pet Supplies,Cat Food
14608,Nood Cage-Free Chicken Recipe With Probiotics ...,Pets,Pet Supplies,Cat Food


,ProductName,level_1,level_2,level_3
0,Johnson's Baby Lotion,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
1,Johnson's Baby Oil,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care
2,SMA Balanced Nutrition From Birth Infant Formula,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food
3,Johnson's Baby Powder,"Baby, Toddler & Kids",Baby Care,Nappies & Changing
4,Vaseline White Petroleum Jelly,"Baby, Toddler & Kids",Baby Care,Nappies & Changing
...,...,...,...,...
12217,Nood Wild-Caught Tuna Recipe In Gravy Premium ...,Pets,Pet Supplies,Cat Food
12218,Nood Grass-Fed Lamb Recipe In Gravy Premium Ca...,Pets,Pet Supplies,Cat Food
12219,Nood Cage-Free Chicken Recipe In Gravy Premium...,Pets,Pet Supplies,Cat Food
12220,Nood Cage-Free Chicken Recipe With Probiotics ...,Pets,Pet Supplies,Cat Food


In [36]:
data.groupby(['level_1'])['ProductName'].count().sort_values(ascending = False).to_frame()

,ProductName
level_1,
Pantry,3750
Personal Care,1658
"Chilled, Frozen & Desserts",1324
Fresh Foods & Bakery,1227
"Beer, Cider & Wine",1190
"Kitchen, Dining & Household",1086
Drinks,887
Pets,454
"Baby, Toddler & Kids",317


In [37]:
data.to_csv(r'product_classification.csv', index = False, header = True)

## Random Over Sampling

In [38]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE  # 随机采样函数 和SMOTE过采样函数

#for tf-idf
data = pd.read_csv('D:/jupyter-workplace/classification_project/data_cleaning/product_classification.csv')
print(data.info())

# 分别获取特征值和标签值
X = data
y = data['level_1']

# 对标签中的变量进行计数统计
print('原始标签数据统计：', Counter(y))

# 随机过采样方法
ros = RandomOverSampler(random_state=0)  # random_state为0（此数字没有特殊含义，可以换成其他数字）使得每次代码运行的结果保持一致
X_oversampled, y_oversampled = ros.fit_resample(X, y)  # 使用原始数据的特征变量和目标变量生成过采样数据集

print('随机过采样处理后：', Counter(y_oversampled))
print(X_oversampled.info()) #随机采样后的特征数据信息

#输出为csv
X_oversampled.to_csv(r'product_classification_ros.csv', index = False, header = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222 entries, 0 to 12221
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductName  12222 non-null  object
 1   level_1      12222 non-null  object
 2   level_2      12222 non-null  object
 3   level_3      12222 non-null  object
dtypes: object(4)
memory usage: 382.1+ KB
None
原始标签数据统计： Counter({'Pantry': 3750, 'Personal Care': 1658, 'Chilled, Frozen & Desserts': 1324, 'Fresh Foods & Bakery': 1227, 'Beer, Cider & Wine': 1190, 'Kitchen, Dining & Household': 1086, 'Drinks': 887, 'Pets': 454, 'Baby, Toddler & Kids': 317, 'Meal Spot': 254, 'Featured': 75})
随机过采样处理后 Counter({'Baby, Toddler & Kids': 3750, 'Beer, Cider & Wine': 3750, 'Chilled, Frozen & Desserts': 3750, 'Drinks': 3750, 'Featured': 3750, 'Fresh Foods & Bakery': 3750, 'Kitchen, Dining & Household': 3750, 'Meal Spot': 3750, 'Pantry': 3750, 'Personal Care': 3750, 'Pets': 3750})
<class 'pandas.core.frame.DataFr

In [39]:
#for W2V
data = pd.read_csv('D:/jupyter-workplace/classification_project/data_cleaning/unique_product.csv')
print(data.info())


# 分别获取特征值和标签值
X = data
y = data['level_1']

# 对标签中的变量进行计数统计
print('原始标签数据统计：', Counter(y))

# 随机过采样方法
ros = RandomOverSampler(random_state=0)  # random_state为0（此数字没有特殊含义，可以换成其他数字）使得每次代码运行的结果保持一致
X_oversampled, y_oversampled = ros.fit_resample(X, y)  # 使用原始数据的特征变量和目标变量生成过采样数据集

print('随机过采样处理后', Counter(y_oversampled))
print(X_oversampled.info()) #随机采样后的特征数据信息

#输出为csv
X_oversampled.to_csv(r'unique_product_ros.csv', index = False, header = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222 entries, 0 to 12221
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            12222 non-null  int64  
 1   ProductName           12222 non-null  object 
 2   Branch                12222 non-null  object 
 3   level_1               12222 non-null  object 
 4   level_2               12222 non-null  object 
 5   level_3               12222 non-null  object 
 6   PricePerItem          12222 non-null  float64
 7   PriceMode             12222 non-null  object 
 8   HasMultiBuyDeal       12222 non-null  bool   
 9   MultiBuyDeal          663 non-null    object 
 10  MultiBuyBasePrice     12222 non-null  float64
 11  MultiBuyPrice         4870 non-null   float64
 12  MultiBuyQuantity      4870 non-null   float64
 13  PromoBadgeImageLabel  5233 non-null   object 
dtypes: bool(1), float64(4), int64(1), object(8)
memory usage: 1.2+ MB
None